# Optimization reduction

In [1]:
from optimize import *


Starting MATLAB engine...
-------------------------
Elapsed: 2.79 sec


In [2]:
result_loaded = load("optims/scenario3/forest_0_100.opt")

In [3]:
from pympler.asizeof import asizeof

# check size
print("TOTAL :", asizeof(result_loaded))
for key in result_loaded.keys():
    print(key, ':', asizeof(result_loaded[key]))

TOTAL : 17759752
x : 328
func_vals : 896
space : 48976
x_iters : 33808
specs : 51448
models : 17672680
fun : 32
random_state : 72


In [4]:
_ = reload(result_loaded, get_params(3), 1, 0, verb=False)

------------------
10: 0.4264
20: 0.3236
30: 0.3236
40: 0.2875
50: 0.2875
60: 0.2875
70: 0.2875
80: 0.2875
90: 0.2875
100: 0.2875
Best score: 0.2875
Best parameters: [0.27132766378910406, 0.001380381361838373, 0.2518454319618381, 74, 410.1861235669267, 62]
Total time elapsed: 89.07 sec



In [5]:
# REDUCE RESULT
del result_loaded.models[:-1]

In [6]:
# check size again
print("TOTAL :", asizeof(result_loaded))
for key in result_loaded.keys():
    print(key, ':', asizeof(result_loaded[key]))

TOTAL : 283800
x : 328
func_vals : 896
space : 48976
x_iters : 33808
specs : 51448
models : 196728
fun : 32
random_state : 72


In [7]:
# Check that it still works
_ = reload(result_loaded, get_params(3), 1, 0, verb=False)

------------------
10: 0.4264
20: 0.3236
30: 0.3236
40: 0.2875
50: 0.2875
60: 0.2875
70: 0.2875
80: 0.2875
90: 0.2875
100: 0.2875
Best score: 0.2875
Best parameters: [0.27132766378910406, 0.001380381361838373, 0.2518454319618381, 74, 410.1861235669267, 62]
Total time elapsed: 89.11 sec



In [8]:
models = [1, 2, 3, 4]
print(id(models))
models = [models[-1]]
print(id(models))
models

510278090696
510278107208


[4]

In [9]:
models = [1, 2, 3, 4]
print(id(models))
del models[:-1]
print(id(models))
models

510230993608
510230993608


[4]

# Reduce while optimizing

In [12]:
result_new = optimize(forest_minimize, get_params(0), 12, 0, verb=False)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
2
1
2
1
------------------
10: 0.0302
Best score: 0.0302
Best parameters: [0.05871694001325902, 0.003936128001463711, 1.2984521643751779, 14, 5.846326121643415, 8]
Total time elapsed: 967.65 sec



# Reduce all saved

In [25]:
from optimize import reduce 
from pathlib import Path
from pympler.asizeof import asizeof
from skopt import load, dump

path = Path("./optims")

for resultpath in path.glob("**/*.opt"):
    result = load(resultpath)
    
    oldsize = asizeof(result)
    reduce(result)
    newsize = asizeof(result)
    
    dump(result, resultpath)
    print("Reduced", resultpath, "by", str(int((oldsize-newsize)/oldsize*100))+"%,", "from", oldsize, "to", newsize)

Reduced optims\scenario0\dummy_0_50.opt by 0%, from 70240 to 70240
Reduced optims\scenario0\dummy_1_50.opt by 0%, from 70240 to 70240
Reduced optims\scenario0\dummy_2_50.opt by 0%, from 70240 to 70240
Reduced optims\scenario0\dummy_3_50.opt by 0%, from 70240 to 70240
Reduced optims\scenario0\dummy_4_50.opt by 0%, from 70272 to 70272
Reduced optims\scenario0\forest_0_50.opt by 96%, from 8033608 to 266504
Reduced optims\scenario0\forest_1_50.opt by 96%, from 8033608 to 266504
Reduced optims\scenario0\forest_2_50.opt by 96%, from 8033608 to 266504
Reduced optims\scenario0\forest_3_50.opt by 96%, from 8033608 to 266504
Reduced optims\scenario0\forest_4_50.opt by 96%, from 8033640 to 266536
Reduced optims\scenario0\gbrt_0_50.opt by 84%, from 560896 to 87232
Reduced optims\scenario0\gbrt_1_50.opt by 84%, from 560896 to 87232
Reduced optims\scenario0\gbrt_2_50.opt by 84%, from 560896 to 87232
Reduced optims\scenario0\gbrt_3_50.opt by 84%, from 560896 to 87232
Reduced optims\scenario0\gbrt_4_5

# See predicted function

In [ ]:
import matlab.engine
from optimize import *
from skopt.acquisition import _gaussian_acquisition
loaded = load("./optims/scenario6/forest_3_50.opt")

In [ ]:
# how optimizer predicts next value:
space = loaded['space']
#X = space.transform(space.rvs(n_samples=result.specs['args']['n_points'], random_state=0))
X = np.array([[10**(i/5-1), 10**(j/10+1)] for i in range(21) for j in range(14)])
values = _gaussian_acquisition(X=X, model=loaded.models[-1], y_opt=np.min(loaded.func_vals),
                               acq_func=loaded.specs['args']['acq_func'],
                               acq_func_kwargs={"xi":loaded.specs['args']['xi'], "kappa":loaded.specs['args']['kappa']})

In [ ]:
# how the actual function works
out = np.zeros((27, 41))
for i in range(27):
    maxIter = 10**(i/20.0+1)
    for j in range(41):
        alpha = 10**(j/20.0-1)
        out[i][j] = loaded.models[-1].predict([[alpha, maxIter]])[0]

savemat('./hyper_opt.mat', mdict={'surface':out, 'x':loaded.x_iters, 'y':loaded.func_vals})